In [1]:
# Import other modules not related to PySpark
import os
import sys 
import seaborn as sns 
import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick  
import matplotlib
from mpl_toolkits.mplot3d import Axes3D 
import math
from IPython.core.interactiveshell import InteractiveShell
from datetime import *
import statistics as stats
# This helps auto print out the items without explixitly using 'print'
InteractiveShell.ast_node_interactivity = "all" 
%matplotlib inline  

In [89]:
df = pd.read_csv('merged4_clean_1.csv')
df.replace('-', np.nan, inplace=True)
df= df.applymap(lambda x: x.replace('�', 'e') if isinstance(x, str) else x)

null_percentage = df.isnull().sum() / len(df) * 100
null_percentage 

Group_ID                                       36.604431
Numero ticket                                   0.000225
Breve description                              40.125070
Date de creation                                0.000000
Duree de premier prise en charge               73.774604
Date de resolution                              0.305028
Max Resolution Date                             0.000000
Type de ticket                                  0.000000
Cause reelle                                   13.851310
Action Name                                     0.000000
Intervenant                                    49.079069
Code Appli                                     34.845349
Groupe                                         36.604431
Priorite                                        0.000000
Beneficiaire                                    0.000000
Date de creation de laction en cours           97.555281
Date de prise en charge de laction en cours    73.774604
Statut                         

In [90]:
# df.drop(columns=['Action Name' , 
#                  'Service Manager HELP','Numero ticket','Solution' , 
#                  'Groupe' , 'Entite du responsable du groupe de la derniere action' , 
#                  'Priorite' , 'Beneficiaire' , 'Date de creation de laction en cours' , 
#                  'Date de prise en charge de laction en cours' , 'Statut' ,'Nom appli' , 'Cause'
#                   ,'Intervenant' ] ,inplace=True )

df = df[['Breve description', 'Date de creation', 'Date de resolution',
       'Max Resolution Date', 'Type de ticket', 'Cause reelle', 'Code Appli' , 'Groupe']]


In [57]:
df.columns 

Index(['Breve description', 'Date de creation', 'Date de resolution',
       'Max Resolution Date', 'Type de ticket', 'Cause reelle', 'Code Appli'],
      dtype='object')

In [91]:
null_percentage = df.isnull().sum() / len(df) * 100 
null_percentage 

Breve description      40.125070
Date de creation        0.000000
Date de resolution      0.305028
Max Resolution Date     0.000000
Type de ticket          0.000000
Cause reelle           13.851310
Code Appli             34.845349
Groupe                 36.604431
dtype: float64

In [92]:
df_resolu = df[df['Date de resolution'].isnull() == False ]
df_resolu = df_resolu[df_resolu['Breve description'].isnull() == False ]


In [97]:
null_percentage = df_resolu.isnull().sum() / len(df_resolu) * 100
null_percentage  

Date de creation        0.000000
Date de resolution      0.000000
Max Resolution Date     0.000000
Type de ticket          0.000000
Cause reelle            6.934360
Code Appli             30.328670
Groupe                 31.831757
Cleaned description     0.000000
dtype: float64

In [62]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string

# # Ensure nltk resources are downloaded
# nltk.download('stopwords')
# nltk.download('wordnet')


In [94]:
def clean_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenize
    tokens = text.split()
    # Remove stop words
    stop_words = set(stopwords.words('french'))
    tokens = [word for word in tokens if word not in stop_words]
    # Lemmatize
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)
# df_resolu['Breve description'].fillna('No description', inplace=True)

# Apply text cleaning
df_resolu['Cleaned description'] = df_resolu['Breve description'].apply(clean_text)


In [95]:
df_resolu.drop(columns=['Breve description' ] , inplace=True)

In [96]:
df_resolu  

,Date de creation,Date de resolution,Max Resolution Date,Type de ticket,Cause reelle,Code Appli,Groupe,Cleaned description
3,2019-05-03 13:28:00,2019-06-27 16:01:00,2019-05-20,Non Qualifie,Cas non prevu - evolution,SFF,SFF - Team SFF,install dll vm capgemini e casablanca
4,2020-01-24 09:50:00,2020-09-13 14:43:00,2020-02-07,Non Qualifie,NaN,SBF,COF ESB - FINANCEMENT METIER,pret tourisme
5,2020-01-27 08:58:00,2020-01-27 13:10:00,2020-02-10,Incident,Erreur de geste deexploitation,SFF,SFF - Team SFF,invsffprd fondsgererestservice v1mandataireres...
6,2020-04-15 08:07:00,2020-04-15 08:20:00,2020-04-29,Incident,Bug applicatif,SBF,COF ESB - FINANCEMENT METIER,calculette esb lance lorsque lon clique dessus
7,2020-05-05 06:33:00,2020-09-13 15:38:00,2020-05-13,Non Qualifie,NaN,SBF,COF ESB - FINANCEMENT METIER,calculette esb dossier i follow cp 4231033 dc ...
...,...,...,...,...,...,...,...,...
444527,2023-07-17 08:59:00,2023-07-18 07:52:00,2023-07-18,Incident,Mauvaise qualite de donnees,PPR,FPA - Anaplan Budget & Charges,j4tap3ka alimentation ttiers
444528,2023-12-09 05:35:00,2023-12-11 14:46:00,2023-12-11,Incident,Erreur de configuration deinfrastructure,CDG,FPA - Anaplan Budget & Charges,j4ksl0ja alim slm pivot pnb
444538,2019-06-05 07:53:00,2019-10-24 15:05:00,2019-06-13,Non Qualifie,Erreur de configuration deinfrastructure,CDG,FPA - Anaplan Budget & Charges,detection nouvelles valeurs referentiel prod pnb
444547,2023-10-13 15:35:00,2023-10-17 07:38:00,2023-10-16,Incident,Anomalie technique,PPR,FPA - Anaplan Budget & Charges,j4tap4ka alimentation referentiel


In [ ]:
df_resolu

In [98]:
df_knn = df_resolu.copy() 

In [99]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()
code_appli_encoded = encoder.fit_transform(df_knn[['Code Appli']].fillna('missing'))

# Convert sparse matrix to dense array
code_appli_encoded_dense = code_appli_encoded.toarray()

# Apply KNN imputation to fill missing values
imputer = KNNImputer(n_neighbors=5)
code_appli_imputed_dense = imputer.fit_transform(code_appli_encoded_dense)

# Decode the imputed values back to original categorical values
code_appli_imputed = encoder.inverse_transform(code_appli_imputed_dense)

# Update the DataFrame with the imputed values
df_knn['Code Appli'] = code_appli_imputed




In [100]:
df_knn.isnull().sum() / len(df_knn) * 100 

Date de creation        0.000000
Date de resolution      0.000000
Max Resolution Date     0.000000
Type de ticket          0.000000
Cause reelle            6.934360
Code Appli              0.000000
Groupe                 31.831757
Cleaned description     0.000000
dtype: float64

In [101]:
df_knn['Cause reelle'].fillna('Pas de cause' , inplace=True) 

In [102]:
df_grp = df_knn.copy() 

In [105]:
df_mat = pd.read_csv('Matrice Incident (1) (1).csv' , sep=";")


In [106]:
code_appli_groups = {}

# Iterate through the DataFrame
for index, row in df_mat.iterrows():
    code_appli = row['Code appli']
    group_1 = row['Groupe 1']
    
    # If 'Code appli' is not already in the dictionary, create a new entry
    if code_appli not in code_appli_groups:
        code_appli_groups[code_appli] = [group_1]
    # If 'Code appli' is already in the dictionary, append the group to the existing list
    else:
        code_appli_groups[code_appli].append(group_1)


In [107]:
code_appli_groups 

{'CLY': ['CLY DSI CLARITY',
  'AMI SIEGE',
  'CLY DSI CLARITY',
  'CLY DSI CLARITY',
  'CLY DSI CLARITY'],
 'PRT': ['AMI SIEGE',
  'INO CASSIOPAE WEB SII METIER',
  'INO CASSIOPAE WEB CONTENTIEUX METIER',
  'INO CASSIOPAE WEB SRR METIER',
  'COF CASSIOPAE WEB SII METIER',
  'COF CASSIOPAE WEB SI2E METIER',
  'COF CASSIOPAE WEB CONTENTIEUX METIER',
  'INO CASSIOPAE WEB METIER',
  'INO CASSIOPAE WEB METIER'],
 'CBM': ['COF CASSIOPAE CBM CONTENTIEUX',
  'COF CASSIOPAE CBM METIER',
  'AMI SIEGE'],
 'SVI': ['AMI SIEGE', 'SVI - Team SVI', 'SVI - Team SVI'],
 'PDE': ['VZT - VizTech',
  'DFCB - Core Factory',
  'DFF - Factory Finance',
  'DFF - Factory Finance',
  'DATA - Team usages DATA',
  'TSE - IAE',
  'DFCB - Core Factory',
  'DFR - Factory Risque',
  'TSE - IAE',
  'DFCB - Core Factory',
  'DATA - Team usages DATA',
  'DFCB - Core Factory',
  'DFCB - Core Factory',
  'DFCB - Core Factory',
  'DFCB - Core Factory'],
 'SLM': ['DAB',
  'DAB',
  'DAB',
  'DAB',
  'SCP - Socle Comptable',
  

In [86]:
df_knn['Date de creation'] = pd.to_datetime(df_knn['Date de creation'])
df_knn['Date de resolution'] = pd.to_datetime(df_knn['Date de resolution'])

# Calculate time difference in days
df_knn['Resolution Time (hours)'] = (df_knn['Date de resolution'] - df_knn['Date de creation']).dt.total_seconds() / 3600



In [87]:
df_knn 

,Date de creation,Date de resolution,Max Resolution Date,Type de ticket,Cause reelle,Code Appli,Cleaned description,Resolution Time,Resolution Time (hours)
3,2019-05-03 13:28:00,2019-06-27 16:01:00,2019-05-20,Non Qualifie,Cas non prevu - evolution,SFF,install dll vm capgemini e casablanca,55,1322.550000
4,2020-01-24 09:50:00,2020-09-13 14:43:00,2020-02-07,Non Qualifie,Pas de cause,SBF,pret tourisme,233,5596.883333
5,2020-01-27 08:58:00,2020-01-27 13:10:00,2020-02-10,Incident,Erreur de geste deexploitation,SFF,invsffprd fondsgererestservice v1mandataireres...,0,4.200000
6,2020-04-15 08:07:00,2020-04-15 08:20:00,2020-04-29,Incident,Bug applicatif,SBF,calculette esb lance lorsque lon clique dessus,0,0.216667
7,2020-05-05 06:33:00,2020-09-13 15:38:00,2020-05-13,Non Qualifie,Pas de cause,SBF,calculette esb dossier i follow cp 4231033 dc ...,131,3153.083333
...,...,...,...,...,...,...,...,...,...
444527,2023-07-17 08:59:00,2023-07-18 07:52:00,2023-07-18,Incident,Mauvaise qualite de donnees,PPR,j4tap3ka alimentation ttiers,0,22.883333
444528,2023-12-09 05:35:00,2023-12-11 14:46:00,2023-12-11,Incident,Erreur de configuration deinfrastructure,CDG,j4ksl0ja alim slm pivot pnb,2,57.183333
444538,2019-06-05 07:53:00,2019-10-24 15:05:00,2019-06-13,Non Qualifie,Erreur de configuration deinfrastructure,CDG,detection nouvelles valeurs referentiel prod pnb,141,3391.200000
444547,2023-10-13 15:35:00,2023-10-17 07:38:00,2023-10-16,Incident,Anomalie technique,PPR,j4tap4ka alimentation referentiel,3,88.050000
